In [2]:
Get-Command *aztag*



CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Cmdlet          Get-AzTag                                          6.11.1     Az.Resources
Cmdlet          New-AzTag                                          6.11.1     Az.Resources
Cmdlet          Remove-AzTag                                       6.11.1     Az.Resources
Cmdlet          Update-AzTag                                       6.11.1     Az.Resources



In [3]:
$tags = get-aztag
$tags | fl


Name  : key1
Count : 9

Name  : value1
Count : 9

Name  : cm-resource-parent
Count : 3

Name  : kostenplaats
Count : 5

Name  : displayName
Count : 1

Name  : ServiceName
Count : 1

Name  : PreferredDc
Count : 1

Name  : CustomerShort
Count : 1

Name  : CloudSuiteVersion
Count : 1

Name  : wvddefaultbaseimage
Count : 1

Name  : CustomerName
Count : 1

Name  : PublicFQDN
Count : 1

Name  : NetbiosName
Count : 1

Name  : VnetCIDR
Count : 1

Name  : InternalFQDN
Count : 1

Name  : settings
Count : 1



In [4]:
$tags[0]



Name Count
---- -----
key1 9



In [5]:
$tag = get-aztag kostenplaats -Detailed
$tag | fl
$tag.Values


Name        : kostenplaats
ValuesTable : 
              Name      Count
              ========  =====
              Customer  5    
              
Count       : 5
Values      : {Customer}


Name     Count
----     -----
Customer 5



In [8]:
$resources = Get-AzResource


In [9]:
$resources.tags


Key                Value
---                -----
key1               key
value1             value
key1               key
value1             value
key1               key
value1             value
key1               key
value1             value
key1               key
value1             value
key1               key
value1             value
cm-resource-parent /subscriptions/0e90be0a-fa33-4d80-8584-1cb472e4e889/resourceGroups/psmngr-avd-r…
cm-resource-parent /subscriptions/0e90be0a-fa33-4d80-8584-1cb472e4e889/resourceGroups/psmngr-avd-r…
cm-resource-parent /subscriptions/0e90be0a-fa33-4d80-8584-1cb472e4e889/resourcegroups/psmngr-avd-r…
kostenplaats       Customer
kostenplaats       Customer
key1               key
value1             value
key1               key
value1             value
key1               key
value1             value
displayName        diagnosticStorageAccount
kostenplaats       Customer
kostenplaats       Customer



In [10]:
$resources


Name              : AzureBackup_psmngr-app1-app1-vm_492582051729521
ResourceGroupName : AzureBackupRG_westeurope_1
ResourceType      : Microsoft.Compute/restorePointCollections
Location          : westeurope
ResourceId        : /subscriptions/0e90be0a-fa33-4d80-8584-1cb472e4e889/resourceGroups/AzureBackupR
                    G_westeurope_1/providers/Microsoft.Compute/restorePointCollections/AzureBackup_
                    psmngr-app1-app1-vm_492582051729521
Tags              : 

Name              : AzureBackup_psmngr-app1-app1-vm_580542982894794
ResourceGroupName : AzureBackupRG_westeurope_1
ResourceType      : Microsoft.Compute/restorePointCollections
Location          : westeurope
ResourceId        : /subscriptions/0e90be0a-fa33-4d80-8584-1cb472e4e889/resourceGroups/AzureBackupR
                    G_westeurope_1/providers/Microsoft.Compute/restorePointCollections/AzureBackup_
                    psmngr-app1-app1-vm_580542982894794
Tags              : 

Name              : AzureBa

In [12]:
class tag {
    # Properties
        $Name
        $Value
        $ResourceId
        $ResourceType
        $ResourceGroupName

    # Constructors
    tag()
    {}
}

function Get-AzTag{
    [CmdletBinding(DefaultParameterSetName='none')]
    param(
        #[Parameter(ParameterSetName="Name")]
        $Name = '*'
        ,
        #[Parameter(ParameterSetName="Name")]
        $Value = '*'
        ,
        #[Parameter(ParameterSetName="Id")]
        $ResourceId
        ,
        #[Parameter(ParameterSetName="ResourceGroup")]
        $ResourceGroupName
        ,
        #[Parameter(ParameterSetName="Type")]
        $ResourceType
    )

    $PSBoundParameters.Remove('Name') | Out-Null
    $PSBoundParameters.Remove('Value') | Out-Null
    $resources = Get-AzResource @PSBoundParameters
    $tagged = $resources.where{$_.tags.keys -ne $null}
    foreach ($t in $tagged) {
        foreach ($tagkey in $t.tags.keys) {
            $tag = [tag]::new()
            $tag.Name = $tagkey
            $tag.ResourceId = $t.ResourceId
            $tag.ResourceGroupName = $t.ResourceGroupName
            $tag.ResourceType = $t.ResourceType
            $tag.Value = $t.tags[$tagkey]
            $tag | where-object {($_.Name -like $name) -and  ($_.Value -like $Value)}
        } 
    }
}

Get-AzTag -value test2


In [11]:
function Set-AzTag{
    [CmdletBinding(DefaultParameterSetName='none',SupportsShouldProcess,ConfirmImpact = 'High')]
    param(
        <#
        maybe put in inputobject parameter for pipeline with a tag object
        [Parameter(ParameterSetName="tag")]
        [tag[]]$tag
        ,
        #>
        #[Parameter(Mandatory,ParameterSetName="Name")]
        $Name
        ,
        #[Parameter(Mandatory,ParameterSetName="Name")]
        $Value
        ,
        #[Parameter(ParameterSetName="Id")]
        $ResourceId
        ,
        #[Parameter(ParameterSetName="ResourceGroup")]
        $ResourceGroupName
        ,
        #[Parameter(ParameterSetName="Type")]
        $ResourceType
    )

    #$PSBoundParameters.Remove('Tag') | Out-Null
    $PSBoundParameters.Remove('Name') | Out-Null
    $PSBoundParameters.Remove('Value') | Out-Null
    $resources = Get-AzResource @PSBoundParameters
    $tags = @{$Name = $Value}
    #If ($PSCmdlet.ParameterSetName('tag')){}
    If ($PSBoundParameters.Keys -notin @('ResourceId','ResourceGroupName','ResourceType')){
        if($PSCmdlet.ShouldProcess("on $($resources.count) resources","setting tag $($name)" )){
            foreach ($resource in $resources) {
                Update-AzTag -ResourceId $resource.id -Tag $tags -Operation Merge
            }
        }
    }else{
        foreach ($resource in $resources) {
            Update-AzTag -ResourceId $resource.id -Tag $tags -Operation Merge
        }
    }
}


Set-AzTag -Name 'test2' -Value 'test2' -ResourceId '/subscriptions/54491160-8439-4f4e-a739-c8cd4b21a042/resourceGroups/cloud-shell-storage-westeurope/providers/Microsoft.Storage/storageAccounts/csb100320003a23915b'



Confirm
Are you sure you want to perform this action?
Performing the operation "zet tag test2" on target "Op 69 resources".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Error: Command cancelled.

In [5]:
function Delete-AzTag{
    [CmdletBinding(SupportsShouldProcess,ConfirmImpact = 'High')]
    param(
        [Parameter(Mandatory)]
        $Name
        ,
        [Parameter(Mandatory)]
        $Value
        ,
        #[Parameter(ParameterSetName="Id")]
        $ResourceId
        ,
        #[Parameter(ParameterSetName="ResourceGroup")]
        $ResourceGroupName
        ,
        #[Parameter(ParameterSetName="Type")]
        $ResourceType
    )

    $PSBoundParameters.Remove('Name') | Out-Null
    $PSBoundParameters.Remove('Value') | Out-Null
    $resources = Get-AzResource @PSBoundParameters
    $tags = @{$Name = $Value}
    
    If ($PSBoundParameters.Keys -notin @('ResourceId','ResourceGroupName','ResourceType')){
        if($PSCmdlet.ShouldProcess("Op $($resources.count) resources","zet tag $($name)" )){
            foreach ($resource in $resources) {
                Update-AzTag -ResourceId $resource.id -Tag $tags -Operation Delete
            }
        }
    }else{
        foreach ($resource in $resources) {
            Update-AzTag -ResourceId $resource.id -Tag $tags -Operation Delete
        }
    }
}


Delete-AzTag -Name 'test' -Value 'test' #-ResourceId '/subscriptions/54491160-8439-4f4e-a739-c8cd4b21a042/resourceGroups/cloud-shell-storage-westeurope/providers/Microsoft.Storage/storageAccounts/csb100320003a23915b'



Confirm
Are you sure you want to perform this action?
Performing the operation "zet tag test" on target "Op 69 resources".
[Y] Yes  [A] Yes to All  [N] No  [L] No to All  [S] Suspend  [?] Help(default is 'Y')

Error: Command cancelled.

In [ ]:
#https://www.geeksforgeeks.org/how-to-update-azure-subscription-tags-using-powershell/

#Switch to Default Azure Subscription
#$SubId = "<add subscriptionID here>"
#Set-AzContext -Subscription "$SubId"

$AzContext = Get-AzContext
$SubId = $AzContext.Subscription.ID

#Add the required tags here
$NewTags = @{
"BusinessArea" = "Finance";
"ITTeam" = "GIT";
"ChargebackTo" = "GIT";
"DataType" = "Internal";
"Environment" = "Production";
"SystemCriticality" = "Critical";

}

#Update Tags on Subscription 
Update-AzTag -ResourceId "/subscriptions/$SubId" -Tag $NewTags -Operation Merge